**Numpy Statevector Simulator**
---


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit
from qiskit.visualization import visualize_transition
print("All modules imported successfully!")

ModuleNotFoundError: ignored

<a name="p1"></a>

---
## **Part 1: Multi-Qubit Circuits and Processing Arbitrary Gates**
---



In [ ]:
def apply_gates(qc,qubit):

  #This function applies the gates of the quantum circuit.

  #We first use the quantum circuit to gain intructions about the gates that are being applied, which helps us implement gates that we haven't
  #hard-coded. We then extract information about the gate we want to apply, and obtain the matrix for that gate. However, this matrix will often
  # be too small to apply to our statevector, so we compose it with a 2^n by 2^n identity matrix so we can make use of the .compose() method's
  # qargs argument, which allows us to specify which qubits to apply the gate to. We extract this information by using the index attribute of the
  #Qubit() object.

  for instruction in qc.data:
    base_matrix = qiskit.quantum_info.operators.Operator(instruction.operation)
    qubits_to_apply_to = [i.index for i in instruction.qubits]
    n_qubit_identity = qiskit.quantum_info.operators.Operator(np.eye(2**qc.num_qubits))
    gate_to_apply = n_qubit_identity.compose(base_matrix,qargs=qubits_to_apply_to)
    qubit=np.dot(gate_to_apply,qubit)
  return qubit

<a name="p2"></a>

---
## **Part 2: Miscellaneous Features**
---

Our simulator now is able to work with any number of qubits, works with complex numbers, and can implement any gate in qiskit! You have created a very flexible simulator, and that is something to be proud of! However, there are plenty of features you could add to make your simulator even more powerful. With the remaining time, try to implement as many as the following features as possible:


*   **Implementing a noise model:**
      - Add the capability to specify different probabilties for different errors to occur. These error rates would correspond to applying an X gate (bit flip error), Z gate (phase flip error), or Y gate (both types of errors) that weren't originally present in the original circuit.
      - Can also add readout errors, decoherence errors, and relaxation errors, but decoherence and relaxation would require a simulation of the time it takes to apply gates.
      - You can then test out how different algorithms work with various error rates!
      - Note that you will need to simulate an error on each qubit. You will then create a single matrix with all of the possible errors after applying each gate. This can be done relatively easy if you create a temporary quantum circuit, as we did in the
*   **Creating an arbitrary initial state**
      - In qiskit, there is a `QuantumCircuit.prepare_state()` method for `QuantumCircuit`s. However, it turns out that this method simply implements a collection of rotation gates (and CX gates, if the state is entangled)!
      - Using the Bloch sphere and trigonometry, find what set of rotations you would need to create a single-qubit state using a collection of rotation gates (rx, ry, and rz in qiskit).
      - However, if the state is entangled, the math gets very messy very quickly ([read more here](https://arxiv.org/pdf/quant-ph/0308006.pdf)). It also requires some sophisticated math to check mathematically if a vector represents an entangled state. Therefore, if the input quantum ciruit has more than one qubit, you can simply use the initial state vector passed as an input as the initial state of the qubit (no need to start from the 0 state).
*  **Modify the graphing of the of the probability histogram to depict phase:**
      - Even though the phase isn't always observable, simulations are able to give us information about phase, and we might as well make use of that.
      - Multiply the statevector by a complex number with magnitude 1 to make the first component real and positive ([read why we can do this here](https://quantum-computing.ibm.com/admin/docs/admin/guide/introducing-qubit-phase)), then convert the complex entries of all the other components into polar form.
      -We can then use the phase angle (the value in the exponent) to generate a unique color. Plot the bar corresponding to each measurable state in this new color to give information about the phase.
      - It may be useful to include a color scale in the comments of your code or on the histogram, if you are able to do that!
*  **Add the option to measure the state in a different basis**
      - We can simulate measurement in other bases by applying a gate that switches the place of the Z axis and the axis we want to measure. For the X axis, the H gate "switches" the X axis and the Z axis. If we end up measuring a 0, we can conclude that before the H gate, the qubit must have been in the + state. If we measure a 1, we can conclude that before the H gate, the qubit must have been in the 0 state. The first step is to find the gate that sends the 0 state to the i state and the 1 state to the -i state.
      - Add an argument to your function that allows the user to specify a basis (X,Y, or Z), and use the method described above to simulate measurement in other bases. Note that you must apply the transformation to every qubit in the quantum circuit. To figure out how to do this using a single matrix, [read this description of using the tensor product to create multi-qubit representations of single qubit gates](https://quantumcomputing.stackexchange.com/a/2271). Note that for vectors/matrices, the tensor product is instead called the Kronecker product.
      - Finally, modify the plotting of the histogram to reflect the possible measurement results.

### **Implementing a Noise Model**

In [ ]:
def apply_gates(qc,qubit,noise):

  #This function applies the gates of the quantum circuit, and also implements the noise model.

  #We first use the quantum circuit to gain intructions about the gates that are being applied, which helps us implement gates that we haven't
  #hard-coded. We create a temporary quantum circuit, which will be used to convert the instruction into an operator. We then randomly pick
  # which qubits to apply the error gates to, and add these to the circuit before converting it to an operator, and applying it to the qubit state.

  for instruction in qc.data:
    qc_temp = qiskit.QuantumCircuit(qc.num_qubits)
    qc_temp.append(instruction)

    x_err = [np.random.choice([0,1],p=[1-noise[0],noise[0]]) for i in range(qc.num_qubits)]
    z_err = [np.random.choice([0,1],p=[1-noise[1],noise[1]]) for i in range(qc.num_qubits)]
    y_err = [np.random.choice([0,1],p=[1-noise[2],noise[2]]) for i in range(qc.num_qubits)]
    for i in range(qc.num_qubits):
      if x_err[i]==1:
        qc_temp.x(i)
      if y_err[i]==1:
        qc_temp.y(i)
      if z_err[i] == 1:
        qc_temp.z(i)
    gate = qiskit.quantum_info.operators.Operator(qc_temp)
    qubit=np.dot(gate,qubit)
  return qubit

### **Creating an Arbitrary Initial State**

In [ ]:
def prepare_state(qc,initial_state):

  #This function is used to prepare an initial state using rotation gates, but only if the initial state is the right size.
  #This function helps solve the problem of preparing an initial state.

  #Checks if the input is a valid input vector
  if initial_state.shape[0]!=2**qc.num_qubits:
      raise ValueError("Invalid Statevector. Statevector must have 2^n components.")

  #The math is too complex for multi-qubit circuits. First, we check if our quantum circuit has only one qubit. If there is,
  #we use the ry gate to adjust the measurement probabilities, then the rz gate to adjust the relative phase. The angles are
  # calulated using the formulas below. Note that all angles on the Bloch sphere are doubled (since, for example, the 0 state
  # and the 1 state are "orthogonal", which normally is a 90 degree angle, but on the Bloch sphere, they are 180 degrees apart).

  if qc.num_qubits ==1:
    qubit = np.array([[1],
                      [0]])
    ry_angle = 2*np.arctan(np.abs(initial_state[1][0])/np.abs(initial_state[0][0]))
    ry_gate = np.array([[np.cos(ry_angle/2),-1*np.sin(ry_angle/2)],[np.sin(ry_angle/2),np.cos(ry_angle/2)]])
    qubit = np.dot(ry_gate,qubit)

    #Here, the goal is to account for relative phase. The way we do this is first by checking that the first component is
    # nonzero. If it is zero, then there is no relative phase to deal with, so we do not need to apply an rz gate. Otherwise,
    # we  use np.angle to calculate the phase angle between the two components. We then rotate the state around the z axis
    # by that angle. Note that for simplicity, we first make it so that the first component is a positive real number. This is
    #equivalent to taking the difference between the phase angle of the second component and the phase angle of the first component.

    if complex(initial_state[0][0])!=complex(0):
      adjusted_qubit_state = (np.abs(initial_state[0][0])/initial_state[0][0])*initial_state
      relative_phase = np.angle(adjusted_qubit_state[1][0])
      rz_angle = relative_phase
      rz_gate = np.array([[np.exp(-1j*rz_angle/2),0],[0,np.exp(1j*rz_angle/2)]])
      qubit=np.dot(rz_gate,qubit)

  #If we have more than one qubit, we can just accept the provided statevector as the initial state of the qubit.

  else:
    qubit = initial_state
  return qubit

### **Modifying the Graph of Probabilities to Depict Phase**

In [ ]:
def create_histogram(qubit,counts):

  #This function plots a histogram of the results, and colors the bars depending on the phase of the state being measured.

  #First, we use np.angle to calculate the phase angle of all of the components. Because this function returns angles
  #between -pi and pi, we add pi to all angles so that instead, our angles range from 0 to 2pi. We then use rgb coloring
  #to pick the color based on the angle. We split up the 2pi radians into 255 levels. An angle of 0 would be 100% green,
  #while an angle of 2pi would correspond to 100% red. The format line converts the rgb level to a hex color, which can be
  #passed to the bar graph.
  print(qubit)
  complex_qubit = np.array([[complex(i[0])] for i in qubit])
  angles = np.angle(qubit)+np.pi
  angles=angles.reshape(-1)
  colors = []
  for angle in angles:
    r=int(np.floor(angle*255/(2*np.pi)))
    g=255-int(np.floor(angle*255/(2*np.pi)))
    color = '#{:02x}{:02x}{:02x}'.format(r, g, 0)
    colors.append(color)

  #This code is extremely similar to the function from lab 10, but we add a color argument to the bar method. Also,
  #note that we need to normalize the heights, so that our plot is not affected by the number of shots

  fig, ax = plt.subplots()
  labels = np.array(list(counts.keys()))
  heights = np.array(list(counts.values()))
  heights=heights/np.sum(heights)
  ax.bar(labels, heights,color=colors)
  ax.set_title("Probability of final state measurements")
  ax.set_xlabel("States")
  ax.set_ylabel("Probability")
  ax.set_ylim([0,1])
  plt.show()

### **Measurement in Different Bases**

In [ ]:
def allowed_final_states(qc,basis):

  #This piece of code establishes, based on the desired measurement basis, the possible measurement results

  if basis=='X':
    possible_results=['+','-']
  elif basis=='Y':
    possible_results=['i','-i']
  elif basis=='Z':
    possible_results=['0','1']

  #This code creates a list of all possible measurement results for combinations of n qubits. This code creates the list in an order that
  #corresponds to the components of the probability vector.
  final_states=[]
  for i in range(2**qc.num_qubits):
    final_state = '{0:0'+str(qc.num_qubits)+'b}'
    final_state = final_state.format(i).replace('0',possible_results[0]).replace('1',possible_results[1])
    final_states.append(final_state)
  return final_states

In [ ]:
def measurement(qc,qubit,basis,final_states):
  if basis == 'Z':
    probability_vector = np.power(np.abs(qubit),2)
    new_qubit=qubit
  elif basis == 'X':
    H = (1/np.sqrt(2))*np.array([[1,1],[1,-1]])
    change_of_basis_matrix = H
    for i in range(qc.num_qubits-1):
      change_of_basis_matrix = np.kron(change_of_basis_matrix,H)
    new_qubit = np.dot(change_of_basis_matrix,qubit)
    probability_vector = np.power(np.abs(new_qubit),2)
  elif basis == 'Y':
    Y_basis = (1/np.sqrt(2))*np.array([[1,1j],[1,-1j]])
    change_of_basis_matrix = Y_basis
    for i in range(qc.num_qubits-1):
      change_of_basis_matrix = np.kron(change_of_basis_matrix,Y_basis)
    new_qubit = np.dot(change_of_basis_matrix,qubit)
    probability_vector = np.power(np.abs(new_qubit),2)
  pick = np.random.choice(final_states,p=probability_vector.reshape(-1))
  return pick, new_qubit

<a name="p3"></a>

---
## **Part 3: Putting it All Together**
---

In [ ]:
def statevector_execute(qc,shots=1000,initial_state=np.array([[0],[0]]),basis="Z",noise=[0,0,0]):

  #This function collects all the features together. First, if we do not specify an initial state, we create a statevector with 2^n components,
  # and a 1 in the first component (which represents the state where all qubits are in the 0 state)

  final_states = []
  if np.all(initial_state==[0]):
      initial_state=np.zeros((2**qc.num_qubits,1))
      initial_state[0][0]=1

  #This function takes in the number of qubits in the quantum circuit and creates a list of all possible measurement results in the desired basis,
  #which aligns with the order of the probability vector

  final_states = allowed_final_states(qc, basis)

  #Creates an empty dictionary, which will be used to keep track of measurement results

  counts = {i:0 for i in final_states}

  #The following code simulates the quantum circuit. Each iteration, we prepare the initial state, apply the gates, then make a measurement.
  #Repeating all of these steps is necessary because each time we run the apply_gates() function, a new set of errors is generated, so we will
  #get different probabilities of measuring different final states. The last line of code increases the counter of the measured state. We then
  #return the counts dictionary and the qubit state of the final measurement (for the phase coloring of the histogram).

  for i in range(shots):
    qubit = prepare_state(qc,initial_state)
    qubit=apply_gates(qc,qubit,noise)
    pick,qubit = measurement(qc,qubit,basis,final_states)
    counts[pick]+=1
  return counts, qubit


#End of notebook
---
© 2023 The Coding School, All rights reserved